<a href="https://colab.research.google.com/github/balakrishna-24/NLP-ML-Chatbot/blob/main/ML_Chat_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas
!pip install nltk


In [ ]:
import zipfile
import pandas as pd

zip_file_path="/content/ap.zip"

with zipfile.ZipFile(zip_file_path,'r') as zip:
  zip.extractall('/content')

excel="/content/customer_support_tickets.csv"
import pandas as pd
df=pd.read_csv(excel)
print(df.head())

   Ticket ID        Customer Name              Customer Email  Customer Age  \
0          1        Marisa Obrien  carrollallison@example.com            32   
1          2         Jessica Rios    clarkeashley@example.com            42   
2          3  Christopher Robbins   gonzalestracy@example.com            48   
3          4     Christina Dillon    bradleyolson@example.org            27   
4          5    Alexander Carroll     bradleymark@example.com            67   

  Customer Gender Product Purchased Date of Purchase      Ticket Type  \
0           Other        GoPro Hero       2021-03-22  Technical issue   
1          Female       LG Smart TV       2021-05-22  Technical issue   
2           Other          Dell XPS       2020-07-14  Technical issue   
3          Female  Microsoft Office       2020-11-13  Billing inquiry   
4          Female  Autodesk AutoCAD       2020-02-04  Billing inquiry   

             Ticket Subject  \
0             Product setup   
1  Peripheral compatibil

In [ ]:
df= pd.read_csv('customer_support_tickets.csv')

ndf= df[['Ticket Description', 'Ticket Type']]
ndf.to_csv('nlpa.csv', index=False)

In [ ]:
# df = pd.read_csv('nlpa.csv')
print("Dataset Shape:", df.shape)

Dataset Shape: (8469, 17)


In [ ]:
df = pd.read_csv('nlpa.csv')
import re

def clean_description(text):
    if pd.isna(text):
        return ""

    text=str(text)
    text=text.lower()
    text=re.sub(r'\{[^}}]+\}', '', text)

    text= re.sub(r'\d+\.\d+\.\d+', '',  text)
    text= re.sub(r'\d+\.\d+\.\d+', '', text)
    text= re.sub(r'please assist', '', text)

    text= re.sub(r'[^a-zA-Z\s]', ' ', text)
    text= ' '.join(text.split())
    return text.strip()

df['cleaned_description']= df['Ticket Description'].apply(clean_description)
cleaned_df= df[['cleaned_description', 'Ticket Type']]
cleaned_df.to_csv('rp.csv', index=False)

In [ ]:
import pandas as pd
import nltk
import os

custom_nltk_dir = './custom_nltk_data'
os.makedirs(custom_nltk_dir, exist_ok=True)
nltk.data.path.append(custom_nltk_dir)
nltk.download('punkt', download_dir=custom_nltk_dir)

[nltk_data] Downloading package punkt to ./custom_nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df = pd.read_csv('rp.csv')

def basic_tokenize(text):
    sentences = text.split('.')
    words = text.split()
    return {
        'sentences': [s.strip() for s in sentences if s.strip()],
        'words': words
    }


df['tokenized'] = df['cleaned_description'].apply(basic_tokenize)
df['sentence_tokens'] = df['tokenized'].apply(lambda x: x['sentences'])
df['word_tokens'] = df['tokenized'].apply(lambda x: x['words'])

df.to_csv('tokenizedt.csv', index=False)

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

df = pd.read_csv('tokenizedt.csv')

import ast
df['word_tokens'] = df['word_tokens'].apply(ast.literal_eval)

def clean_text(tokens):
   return [word.lower() for word in tokens if word.isalpha()]

stop_words = set(stopwords.words('english'))
def remove_stopwords(tokens):
   return [word for word in tokens if word not in stop_words]

lemmatizer = WordNetLemmatizer()
def lemmatize_text(tokens):
   return [lemmatizer.lemmatize(word) for word in tokens]


df['cleaned_tokens'] = df['word_tokens'].apply(clean_text)
df['tokens_no_stop'] = df['cleaned_tokens'].apply(remove_stopwords)
df['lemmatized'] = df['tokens_no_stop'].apply(lemmatize_text)

df.to_csv('new_text.csv', index=False)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
import pandas as pd
from gensim.models import Word2Vec
import numpy as np
import ast

df= pd.read_csv('new_text.csv')

df['lemmatized'] = df['lemmatized'].apply(ast.literal_eval)

model = Word2Vec(sentences=df['lemmatized'],
                vector_size=200,
                window=5,
                min_count=2,
                epochs=5,
                workers=4)

def get_doc_vector(tokens):
    vec = np.zeros(200)
    count = 0
    for word in tokens:
        if word in model.wv:
            vec += model.wv[word]
            count += 1
    return vec / count if count != 0 else vec


df['document_vectors'] = df['lemmatized'].apply(get_doc_vector)

model.save("word2vec_model.model")
df.to_csv('embedded_data.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.model_selection import train_test_split


df = pd.read_csv('embedded_data.csv')

def string_to_array(vector_string):
    try:
        nums = vector_string.strip('[]').split()
        return np.array([float(num) for num in nums])
    except:
        return np.zeros(100)

df['document_vectors'] = df['document_vectors'].apply(string_to_array)

X = np.stack(df['document_vectors'].values)
y = df['Ticket Type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)



In [ ]:
from sklearn.ensemble import RandomForestClassifier
import joblib

clf = RandomForestClassifier()
clf.fit(X_train, y_train)


joblib.dump(clf, 'random_forest_model.joblib')
print("Model saved successfully")

accuracy = clf.score(X_test, y_test)
print(f"Accuracy: {accuracy}")

Model saved successfully
Accuracy: 0.20838252656434475


In [ ]:
print("Class distribution in training data:")
print(y_train.value_counts())

Class distribution in training data:
Ticket Type
Technical issue         1410
Refund request          1390
Cancellation request    1356
Billing inquiry         1317
Product inquiry         1302
Name: count, dtype: int64


In [ ]:
def simple_preprocess(text):
    tokens = text.lower().split()
    tokens = [word.strip('.,!?()[]{}:;') for word in tokens]
    tokens = [word for word in tokens if word.isalpha()]
    return tokens

def get_doc_vector(tokens):
    vec = np.zeros(200)
    count = 0
    for word in tokens:
        if word in model.wv:
            vec += model.wv[word]
            count += 1
    return vec / count if count != 0 else vec


from gensim.models import Word2Vec
model = Word2Vec.load("word2vec_model.model")

In [ ]:
test_texts = [
    "I need a refund for my purchase",
    "Device not turning on after charging",
    "Cancel my subscription immediately",
    "Technical error when launching app"
]

for text in test_texts:
    tokens = simple_preprocess(text)
    vector = get_doc_vector(tokens)
    prediction = clf.predict([vector])
    print(f"Text: {text}\nPredicted Type: {prediction[0]}\n")

Text: I need a refund for my purchase
Predicted Type: Refund request

Text: Device not turning on after charging
Predicted Type: Billing inquiry

Text: Cancel my subscription immediately
Predicted Type: Cancellation request

Text: Technical error when launching app
Predicted Type: Refund request



In [ ]:
import pandas as pd

df = pd.read_csv('customer_support_tickets.csv')
df_time = df[['First Response Time', 'Time to Resolution']]
df_time.to_csv('resolution_time_only.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import joblib

df = pd.read_csv('resolution_time_only.csv')

df['First Response Time'] = pd.to_datetime(df['First Response Time']).astype(np.int64) // 10**9
df['Time to Resolution'] = pd.to_datetime(df['Time to Resolution']).astype(np.int64) // 10**9

df['Resolution_Time_Minutes'] = (df['Time to Resolution'] - df['First Response Time'])/60
df = df.dropna()

X = df[['First Response Time']]
y = df['Resolution_Time_Minutes']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = RandomForestRegressor()
model.fit(X_train, y_train)

accuracy = model.score(X_test, y_test)
print(f"Model Accuracy: {accuracy}")

joblib.dump(model, 'resolution_time_model.joblib')







Model Accuracy: 0.06843307873601723


['resolution_time_model.joblib']

In [ ]:
import joblib
import numpy as np
import pandas as pd

model = joblib.load('resolution_time_model.joblib')

def predict_resolution_time(first_response_time):
   first_response_timestamp = pd.to_datetime(first_response_time).timestamp()
   prediction = model.predict([[first_response_timestamp]])
   return prediction[0]

test_time = "2024-02-04 10:00:00"
predicted_minutes = predict_resolution_time(test_time)
print(f"Predicted resolution time in minutes: {predicted_minutes}")

test_times = [
   "2024-02-04 09:00:00",
   "2024-02-04 14:30:00",
   "2024-02-04 18:45:00"
]

for time in test_times:
   prediction = predict_resolution_time(time)
   print(f"Time: {time} -> Predicted minutes: {prediction}")

Predicted resolution time in minutes: -56970221.03052776
Time: 2024-02-04 09:00:00 -> Predicted minutes: -56970221.03052776
Time: 2024-02-04 14:30:00 -> Predicted minutes: -56970221.03052776
Time: 2024-02-04 18:45:00 -> Predicted minutes: -56970221.03052776


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
